# Practica uno

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica solo se usará la librería de numpy.

In [ ]:
import numpy as np

## Ejercicio 1



In [ ]:
class CodificadorEtiqueta:
	def __init__(self):
		self.codificacion = None
		self.decodificacion = None

	def inicializar(self, y: list):
		valores_unicos = np.unique(y)
		self.codificacion={valor: i for i, valor in enumerate(valores_unicos)}
		self.decodificacion={i: valor for i, valor in enumerate(valores_unicos)}
		print(self.codificacion)
		print(self.decodificacion)

	def codificar(self, y: list):
		return [self.codificacion[valor] for valor in y]

	def decodificar(self, y: list):
		return [self.decodificacion[valor] for valor in y]

In [ ]:
codificador = CodificadorEtiqueta()
codificador.inicializar(['a', 'b', 'c', 'a'])

In [ ]:
codificador.codificar(['a', 'b', 'c', 'a'])
codificador.decodificar([0, 1, 2, 0, 2, 2, 2, 1, 0, 0])

## Ejercicio 2

In [ ]:
class EscalarValores:
	def __init__(self):
		self.max = None
		self.min = None
		self.rangos  = None
	
	def inicializar(self, x: np.ndarray, min: float=-1.0, max: float=1) -> None:
		self.max = max
		self.min = min
		mins = np.min(x, axis=0)
		maxs = np.max(x, axis=0)
		self.rangos = np.vstack((mins, maxs))

	def escalar(self, x: np.ndarray) -> np.ndarray:
		if (self.rangos is None or self.max is None or self.min is None):
			raise Exception("No se ha inicializado el escalador")
		data_maxs = np.max(x, axis=0)
		data_mins = np.min(x, axis=0)
		if (np.any(data_maxs > self.rangos[1]) or np.any(data_mins < self.rangos[0])):
			raise Exception("Valores fuera de rango")
		return (x - self.rangos[0]) / (self.rangos[1] - self.rangos[0]) * (self.max - self.min) + self.min

	def escalar_inv(self, x: np.ndarray) -> np.ndarray:
		if (self.rangos is None or self.max is None or self.min is None):
			raise Exception("No se ha inicializado el escalador")
		if(np.any(x > self.max) or np.any(x < self.min)):
			raise Exception("Valores fuera de rango")
		return (x - self.min) / (self.max - self.min) * (self.rangos[1] - self.rangos[0]) + self.rangos[0]

In [ ]:
xs = np.array([
	[0, 10],
	[1, 20]
])

m = EscalarValores()

try:
	m.escalar(xs)
except Exception as e:
	print(e)

m.inicializar(xs)
print(m.escalar(xs))
print(m.escalar(np.array([[0.5, 15], [0.7, 12]])))
print(m.escalar_inv(m.escalar(xs)))

try:
	m.escalar(np.array([[1.2, 21]]))
except Exception as e:
	print(e)

## Ejercicio 3

In [ ]:
def divide_entrenamiento_test(
	x: np.ndarray,
	y: np.ndarray | None=None,
	tam_train: float=0.7,
	semilla: int | None=None,
	mezclar: bool=True
) -> list:
	total_size = x.shape[0]
	if (y is not None and total_size != y.shape[0]):
		raise Exception("Las dimensiones de x e y no coinciden")
	if (tam_train <= 0 or tam_train >= 1):
		raise Exception("El tamaño de entrenamiento debe estar entre 0 y 1")
	has_y = y is not None
	train_size = int(total_size * tam_train)
	transform_mat = x.copy()
	if(has_y):
		transform_mat = np.hstack((x, np.array([y]).T))
	if(mezclar):
		np.random.seed(semilla)
		np.random.shuffle(transform_mat)
	
	train = transform_mat[:train_size]
	test = transform_mat[train_size:]
	if(has_y):
		return [train[:, :-1], test[:, :-1], train[:, -1], test[:, -1]]
	else:
		return [train, test]


In [ ]:
x = np.array([
  [1, 2, 3],
  [4, 5, 6],
  [7, 8, 9],
  [10, 11, 12],
  [13, 14, 15],
  [16, 17, 18],
  [19, 20, 21],
])

y = np.array([10, 20, 30, 40, 50, 60, 70])

xa, xb, ya, yb = divide_entrenamiento_test(x, y, 0.5, mezclar=True, semilla=42)

print(xa)
print()
print(ya)
print()
print(xb)
print()
print(yb)